In [1]:
import csv
import datetime as dt

from humanize import naturaldelta
from pytz import timezone
import pytz
utc = pytz.utc
pacific = timezone('US/Pacific')

In [ ]:
with open('apple-oosa.csv', 'w') as of:
    writer = csv.writer(of, delimiter=',', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(["Bike's name", 'Alert Started', 'Alert Stopped', 'Duration', "Renter's email"])
    for alert in (Alert.objects.filter(alert_type=Alert.RIDE_OUTSIDE_SERVICE_AREA,
                                       organization__id=15,
                                       created__gte=utc.localize(dt.datetime(2017, 6, 20, 0, 0, 0)))
                                .annotate(duration_outside=F('modified') - F('created'))
                                .order_by('-duration_outside', 'bicycles__name')):
        user = alert.causality.rental_sessions.filter(modified__lt=alert.created, state='closed').latest().user
        writer.writerow((alert.causality.name,
                         alert.created.astimezone(pacific).isoformat(),
                         alert.modified.astimezone(pacific).isoformat(),
                         naturaldelta(alert.duration_outside),
                         user.email))
              